In [18]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/Datasets/phoenix data 4.2-4.3.csv")

In [22]:
Abhi.head(5)

,CBO,NOC,WMC,RFC,SLOC,LCOM,DIT,Change
0,5,0,3,16,44,0,0,yes
1,9,0,10,23,309,70,1,yes
2,31,1,4,17,154,58,1,yes
3,2,0,3,16,15,33,1,no
4,27,0,23,36,544,78,0,yes


In [23]:
Abhi['Change'] = Abhi['Change'].map({'yes': 1, 'no': 0})

In [24]:
Abhi.shape

(1100, 8)

In [25]:
Abhi.head()

,CBO,NOC,WMC,RFC,SLOC,LCOM,DIT,Change
0,5,0,3,16,44,0,0,1
1,9,0,10,23,309,70,1,1
2,31,1,4,17,154,58,1,1
3,2,0,3,16,15,33,1,0
4,27,0,23,36,544,78,0,1


In [26]:
X = Abhi.drop('Change', axis=1)
y = Abhi['Change']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [28]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = DecisionTreeClassifier()
  #level1 = DecisionTreeClassifier()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [29]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [30]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [31]:
models = get_models()

In [32]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.405 (0.491)


In [33]:
print(y_pred)

[0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0
 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1
 1 1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0
 1 0 1 1 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 1 1
 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0]


In [34]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[85 47]
 [46 42]]


In [35]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.5606060606060607


In [36]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.12098839122454864


In [37]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5606060606060606


In [38]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.5543777689841122


In [39]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.64      0.65       132
           1       0.47      0.48      0.47        88

    accuracy                           0.58       220
   macro avg       0.56      0.56      0.56       220
weighted avg       0.58      0.58      0.58       220

